Upload Dataset from local drive

In [ ]:
from google.colab import files
uploaded = files.upload()


1. Read the dataset by passing it as a parameter to the read_csv() in pandas to get the dataframe.

In [ ]:
import pandas as pd
df = pd.read_csv("/content/OhioQuar.csv")
print(df)

       API_WellNumber  Production_Year  QUARTER  \
0        3.400000e+13             2020        1   
1        3.410000e+13             2020        1   
2        3.410000e+13             2020        1   
3        3.410000e+13             2020        1   
4        3.410000e+13             2020        1   
...               ...              ...      ...   
10622    3.400000e+13             2020        4   
10623    3.400000e+13             2020        4   
10624    3.410000e+13             2020        4   
10625    3.410000e+13             2020        4   
10626    3.410000e+13             2020        4   

                          OWNER_NAME    COUNTY     TOWNSHIP  \
0      ANTERO RESOURCES  CORPORATION   BELMONT     KIRKWOOD   
1      ANTERO RESOURCES  CORPORATION  GUERNSEY     MILLWOOD   
2      ANTERO RESOURCES  CORPORATION  GUERNSEY     MILLWOOD   
3      ANTERO RESOURCES  CORPORATION  GUERNSEY     MILLWOOD   
4      ANTERO RESOURCES  CORPORATION  GUERNSEY     MILLWOOD   
...      

Crosscheck column names

In [ ]:
print(df.columns)


Index(['API_WellNumber', 'Production_Year', 'QUARTER', 'OWNER_NAME', 'COUNTY',
       'TOWNSHIP', 'WELL_NAME', 'WELL_NUMBER', 'OIL', 'GAS', 'BRINE', 'DAYS'],
      dtype='object')


2. To calculate the annual data for oil, gas, and brine based on the API_WellNumber, we can group the data by the API_WellNumber and then sum the quarterly production values for each category.

Here's the code to perform the required calculation:

This code will group the DataFrame df by the 'API_WellNumber' column and calculate the sum of 'OIL', 'GAS', and 'BRINE' for each well. The resulting DataFrame, annual_data, will contain the annual production values for each category.

In [ ]:
annual_data = df.groupby('API_WellNumber')['OIL', 'GAS', 'BRINE'].sum()
print(annual_data)

                                                              OIL  \
API_WellNumber                                                      
3.400000e+13    0000229003950000000003490000000000032809200006...   
3.410000e+13    1921031071301582922204236165777488377347544576...   
3.420000e+13    2732500001,93345102,056940499000367204004881,2...   

                                                              GAS  \
API_WellNumber                                                      
3.400000e+13    081,17392,7411,38,57125,01002,76,25943,03066,0...   
3.410000e+13    32,29327,27628,39331,52131,33446,77362,98021,9...   
3.420000e+13    85,21714,809001425,12916,293013,16811,49320,75...   

                                                            BRINE  
API_WellNumber                                                     
3.400000e+13    01,41560875352401,3989033522351,7794215123222,...  
3.410000e+13    55242254255324209708546931,2158414931,1215341,...  
3.420000e+13    1,638730005171980166

<ipython-input-20-d9277edba200>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  annual_data = df.groupby('API_WellNumber')['OIL', 'GAS', 'BRINE'].sum()


3. To load the calculated annual data into a local SQLite database using Python, we need to install the sqlite3 package.

This code creates a table called annual_data in the database with columns for API_WellNumber, OIL, GAS, and BRINE. It then iterates over the annual data DataFrame and inserts the values into the table.

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('Ohio_Database.db')  # Replace 'your_database_name.db' with the desired database name

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Create a table for the annual data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS annual_data (
        API_WellNumber INTEGER PRIMARY KEY,
        OIL INTEGER,
        GAS INTEGER,
        BRINE INTEGER
    )
''')

# Insert the annual data into the table
for index, row in annual_data.iterrows():
    api_wellnumber = int(index)
    oil = row['OIL']
    gas = row['GAS']
    brine = row['BRINE']
    
    cursor.execute('INSERT OR IGNORE INTO annual_data (API_WellNumber, OIL, GAS, BRINE) VALUES (?, ?, ?, ?)',
               (api_wellnumber, oil, gas, brine))

# Commit the changes and close the connection
conn.commit()
conn.close()



4. Now as we run the Flask app, we can send a GET request to http://localhost:8080/data?well=34059242540000, and it will return the annual data for the specified well in the JSON format as mentioned in your example.

In [ ]:
from flask import Flask, jsonify, request
import sqlite3

# Create a Flask app
app = Flask(__name__)

# Define the route for retrieving annual data
@app.route('/data', methods=['GET'])
def get_annual_data():
    # Extract the well API_WellNumber from the query parameters
    well_api_number = request.args.get('well')
    
    # Connect to the SQLite database
    conn = sqlite3.connect('Ohio_Database.db')

    # Create a cursor object to execute SQL commands
    cursor = conn.cursor()

    # Retrieve the annual data for the specified well API_WellNumber
    cursor.execute("SELECT OIL, GAS, BRINE FROM annual_data WHERE API_WellNumber = ?", (well_api_number,))
    result = cursor.fetchone()

    # Close the connection to the database
    conn.close()

    # If the well API_WellNumber is found in the database, return the annual data
    if result is not None:
        oil, gas, brine = result
        annual_data = {
            'oil': oil,
            'gas': gas,
            'brine': brine
        }
        return jsonify(annual_data)
    
    # If the well API_WellNumber is not found in the database, return an error message
    error_message = {
        'error': f"No annual data found for well API_WellNumber {well_api_number}"
    }
    return jsonify(error_message), 404

# Run the Flask app on port 8080
if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
